In [30]:
# bidaf ans
# 1.使用|分割可能有问题
# 2.还未采用threshold卡answer
import json
import os
# Bidaf EQnA
# file = '/home/t-honli/bi-att-flow/out/EQnA/03-07-2017/answer/test-020000.json'
# R-Net EQnA
file = '/home/t-honli/bi-att-flow/r-net/dev-v1.1-MultiAnswer.json'
bidaf = json.load(open(file, "r"))

In [31]:
# prod model 
import pandas as pd
col_names = ['id', 'Query', 'Url', 'Answer', 'AnswerTokenList', 'ParaseSpan', 'phrase', 'Label', 'Probability']
prod = pd.read_csv('/home/t-honli/data/EQnA/EQnA_Highlighting_Test_ProdModel.tsv', header=None, sep='\t', names=col_names, dtype=str).fillna('')

print (len(prod))

# 根据answer，query，url生成hash_id
import hashlib
def GetHashCode(context):
    hash = hashlib.md5()
    hash.update(context.encode('utf-8'))
    return hash.hexdigest()
prod['hash_id'] = prod.apply(lambda row: GetHashCode(row['Answer'] + ' ' + row['Query'] + ' ' + row['Url']), axis=1)

# 增加bidaf的标签和权重，暂时取了3个answer,只要bidaf的phrase覆盖了true phrase就返回score，第一个answer的score是1
def genereateBidafLabel(row):
    # 如果bidaf里面没有这个id，返回0
    if row['hash_id'] not in bidaf:
        return 0
    phrases = bidaf[row['hash_id']].split('|||')
    scores = bidaf['scores'][row['hash_id']].split('|||')
    cnt = 0
    for phrase, score in zip(phrases, scores):
        if row['phrase'] in phrase or phrase in row['phrase']:  # interact(cover)
            if cnt == 0:
                return 1
            else:
                return score
        cnt += 1
    # 没有cover，返回0
    return 0

prod['bidaf'] = prod.apply(genereateBidafLabel, axis=1)
prod = prod[prod.apply(lambda row: len(row['phrase'])<32, axis=1)]
print (len(prod))
prod.head(5)

15560
15454


,id,Query,Url,Answer,AnswerTokenList,ParaseSpan,phrase,Label,Probability,hash_id,bidaf
0,00021900a221a0b2e5c9a923c4f13813,where does the water from mississippi's rivers...,http://en.wikipedia.org/wiki/Mississippi_River,"In the United States, the Mississippi River dr...","[""In"",""the"",""United"",""States"","","",""the"",""Missi...",79:79,rivers,0,0.01860236,9b6bef3c4ce179302bbad6ddf9b98832,0
1,0004dd703e44213707c923e548627774,good foods for pregnant women,http://www.netdoctor.co.uk/health_advice/facts...,"During pregnancy, a woman's body needs more ir...","[""During"",""pregnancy"","","",""a"",""woman"",""'s"",""bo...",4:4,woman,0,0.1246661,ac6273ec9adf9778fb33f09501f43874,0
2,00052a47dccf1667995ebffb3019b565,what continent is belize on,http://www.belize.com/where-is-belize,Belize is located on the Caribbean coast of no...,"[""Belize"",""is"",""located"",""on"",""the"",""Caribbean...",19:19,88°45,0,0.02859148,2110154eb61df6e89f85c33c204492e0,0
3,00099640760fac2092756da3714c60a9,when was tom seaver born,http://www.baseball-almanac.com/players/player...,"Tom Seaver was born on Friday, November 17, 19...","[""Tom"",""Seaver"",""was"",""born"",""on"",""Friday"","",""...",17:17,Seaver,0,0.001268676,a272a8fdd6b0e63d75e39f732639c31f,1
4,000e8ade636d16933ddc53486a59a976,disease caused by deer ticks,http://www.niaid.nih.gov/topics/tickborne/Page...,"Ticks transmit ehrlichiosis and anaplasmosis, ...","[""Ticks"",""transmit"",""ehrlichiosis"",""and"",""anap...",0:0,Ticks,0,0.008481221,54ee3a3b1b57174ee8ee50625d20ee74,0.0282894428819418


In [32]:
prod.head(20)

,id,Query,Url,Answer,AnswerTokenList,ParaseSpan,phrase,Label,Probability,hash_id,bidaf
0,00021900a221a0b2e5c9a923c4f13813,where does the water from mississippi's rivers...,http://en.wikipedia.org/wiki/Mississippi_River,"In the United States, the Mississippi River dr...","[""In"",""the"",""United"",""States"","","",""the"",""Missi...",79:79,rivers,0,0.01860236,9b6bef3c4ce179302bbad6ddf9b98832,0
1,0004dd703e44213707c923e548627774,good foods for pregnant women,http://www.netdoctor.co.uk/health_advice/facts...,"During pregnancy, a woman's body needs more ir...","[""During"",""pregnancy"","","",""a"",""woman"",""'s"",""bo...",4:4,woman,0,0.1246661,ac6273ec9adf9778fb33f09501f43874,0
2,00052a47dccf1667995ebffb3019b565,what continent is belize on,http://www.belize.com/where-is-belize,Belize is located on the Caribbean coast of no...,"[""Belize"",""is"",""located"",""on"",""the"",""Caribbean...",19:19,88°45,0,0.02859148,2110154eb61df6e89f85c33c204492e0,0
3,00099640760fac2092756da3714c60a9,when was tom seaver born,http://www.baseball-almanac.com/players/player...,"Tom Seaver was born on Friday, November 17, 19...","[""Tom"",""Seaver"",""was"",""born"",""on"",""Friday"","",""...",17:17,Seaver,0,0.001268676,a272a8fdd6b0e63d75e39f732639c31f,1
4,000e8ade636d16933ddc53486a59a976,disease caused by deer ticks,http://www.niaid.nih.gov/topics/tickborne/Page...,"Ticks transmit ehrlichiosis and anaplasmosis, ...","[""Ticks"",""transmit"",""ehrlichiosis"",""and"",""anap...",0:0,Ticks,0,0.008481221,54ee3a3b1b57174ee8ee50625d20ee74,0.0282894428819418
5,000f65f7bfa91dc8727d60b3825a53a3,when was ming na born,http://en.wikipedia.org/wiki/Ming-Na,Ming-Na Wen (Chinese: 温明娜; pinyin: Wēn Míngnà)...,"[""Ming"",""-"",""Na"",""Wen"",""("",""Chinese"","":"",""温明娜""...",16:19,"November 20 , 1963",1,0.9577994,37af8df7f2ead5fe71e3f5c403cbb38a,1
6,000fd09205222304bdc584f269a671e2,what is the brightest planet in the sky,http://www.universetoday.com/33674/the-brighte...,The only objects brighter than Venus are the M...,"[""The"",""only"",""objects"",""brighter"",""than"",""Ven...",13:13,Sirius,0,0.482689,7db710fe1fee79cd95045df6a19d4d45,1
7,0011f0f28bb6eb335a64bce9ffbb977d,where is eareckson air station ak,http://en.wikipedia.org/wiki/Eareckson_Air_Sta...,Eareckson Air Station is located on the wester...,"[""Eareckson"",""Air"",""Station"",""is"",""located"",""o...",19:19,Attu,0,0.8296606,a8eecae3c7565ee840f66e3661bdf25d,1
8,0015dd4c1212265294691e1ca52d4169,age of consent in jamaica 16,http://www.jamaicaobserver.com/columns/The-age...,An individual who engages in sex with someone ...,"[""An"",""individual"",""who"",""engages"",""in"",""sex"",...",19:19,abuse,0,0.004436283,6ff14454c5fe88ad27dd90f98705dccf,0
9,0016e626a95e1598881cbf1cbdbf3788,where does the clause separation of church and...,http://en.wikipedia.org/wiki/Separation_of_chu...,"Separation of church and state (sometimes ""wal...","[""\"""",""Separation"",""of"",""church"",""and"",""state""...",37:38,the Establishment,0,0.09433173,2a8188f055bbae54822ee6c3013382b6,0


In [33]:
# prod.to_csv('./data/bidaf_prodlabel.tsv', header=True, index=None, sep='\t') bidaf
prod.to_csv('./data/r-net_prodlabel.tsv', header=True, index=None, sep='\t') # r-net

In [34]:
# add score
# SampleAndCompactMulti.ipynb add threshold
# generate_prodformat_label.ipynb change bidaf data to prod format